# [실습3] 데이터 변환하기 2

## 데이터 불러오기

이전 실습에서 우리는 데이터의 형 변환을 통해 숫자와 날짜 데이터들의 형태를 변환해주고, 날짜 데이터를 활용해 연, 월, 일, 요일 컬럼을 생성하였습니다. 지난 실습을 통해 정제된 데이터를 불러옵니다.

In [4]:
import numpy as np
import pandas as pd

df=pd.read_csv("./data/seoul_park02.csv")
df.head()

,날짜,공휴일,날씨,유료합계,어른,청소년,어린이,외국인,단체,무료합계,총계,연,월,일,요일
0,2016-01-01,O,구름 조금,3359,2799,141.0,419,47,0,1023,4382,2016,1,1,4
1,2016-01-02,O,구름 많음,5173,4370,203.0,600,100,111,2092,7265,2016,1,2,5
2,2016-01-03,O,구름 많음,3008,2571,128.0,309,91,0,1549,4557,2016,1,3,6
3,2016-01-04,X,구름 많음,890,602,NaN,235,51,223,800,1690,2016,1,4,0
4,2016-01-05,X,구름 많음,416,319,35.0,62,43,47,840,1256,2016,1,5,1


요일 컬럼이 글자가 아닌 숫자로 기록되어 있는 것을 확인할 수 있습니다. 이대로도 데이터 분석을 진행할 수는 있지만, 알아보기가 조금 어렵습니다. 

## 열 전체 변환: map()

이 숫자들은 0부터 6까지, 월요일부터 일요일까지 숫자가 매칭되어있습니다. 이 정보를 활용하여 "요일" 컬럼 전체 데이터를 변환하고자 `map()`을 사용합니다.

먼저 `map()`에 변환 정보를 입력해주기 위해 숫자를 키로, 그에 대응하는 요일 글자를 값으로 가지는 `week` 딕셔너리를 생성합니다.

In [5]:
week={0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토',6:'일'}

이제 `map()`과 요일 정보가 담긴 `week` 딕셔너리를 사용해서 `df`의 "요일" 컬럼을 변환합니다.

In [9]:
df['요일']=df['요일'].map(week)
df.head()

,날짜,공휴일,날씨,유료합계,어른,청소년,어린이,외국인,단체,무료합계,총계,연,월,일,요일
0,2016-01-01,O,구름 조금,3359,2799,141.0,419,47,0,1023,4382,2016,1,1,NaN
1,2016-01-02,O,구름 많음,5173,4370,203.0,600,100,111,2092,7265,2016,1,2,NaN
2,2016-01-03,O,구름 많음,3008,2571,128.0,309,91,0,1549,4557,2016,1,3,NaN
3,2016-01-04,X,구름 많음,890,602,NaN,235,51,223,800,1690,2016,1,4,NaN
4,2016-01-05,X,구름 많음,416,319,35.0,62,43,47,840,1256,2016,1,5,NaN


요일 컬럼의 데이터들이 딕셔너리에 맞춰 변환된 것을 확인할 수 있습니다.

In [13]:
## 예시 ##
df[['어른', '청소년', '어린이']].map(lambda x: x * 100)

,어른,청소년,어린이
0,279900,14100.0,41900
1,437000,20300.0,60000
2,257100,12800.0,30900
3,60200,NaN,23500
4,31900,3500.0,6200
...,...,...,...
1081,46400,1000.0,3000
1082,68700,4600.0,2800
1083,144700,12000.0,7700
1084,132600,4400.0,16900



## 데이터에 함수 적용: apply()

이번엔 "날씨" 컬럼을 살펴보겠습니다.

In [14]:
df['유료합계'] * 100

0       335900
1       517300
2       300800
3        89000
4        41600
         ...  
1081     50400
1082     76100
1083    164400
1084    153900
1085    306100
Name: 유료합계, Length: 1086, dtype: int64

날씨 컬럼 데이터에는  '눈', '비', '눈/비' 가 있습니다. 날씨와 입장객수의 관계는 매우 밀접하지만,    
이것은 어디까지나 날씨가 맑은지 흐린지가 중요할 뿐 비인지 눈인지는 크게 중요하지 않습니다.    
그렇기 때문에 세 종류의 데이터를 모두 '눈/비'로 통일해서 합치고자 합니다.   

먼저 '눈', '비' 를 입력받으면 '눈/비'로 바꿔주는 함수를 선언합니다.

In [15]:
# 함수를 쓰는 방법
def weather(e):
    if e=='눈' or e=='비':
        return '눈/비'
    else:
        return e

이제 이 함수를 컬럼 전체에 적용하기 위해 `apply()`를 사용합니다. `apply()`는 특정 컬럼 혹은 데이터프레임 전체에 특정 함수를 적용할 때 사용 가능합니다. 

`df`의 "날씨" 컬럼에 `weather` 함수를 적용하려면 아래와 같이 사용할 수 있습니다.

In [16]:
df["날씨"]=df["날씨"].apply(weather)

In [17]:
df["날씨"].value_counts()

날씨
구름 많음    277
구름 조금    236
맑음       222
눈/비      111
흐림       100
Name: count, dtype: int64

"날짜" 컬럼의 데이터에서 '눈', '비'가 '눈/비'로 통일되어 111개가 되었음을 확인할 수 있습니다.

이렇게 원하는 함수를 정의해서 `apply()`를 활용해 적용하는 과정은 굉장히 많이 사용되게 됩니다. 하지만 다양한 변환이 필요할수록 매번 함수를 만들고 사용해야하고, 이는 코드를 지저분하게 만듭니다.

파이썬에서는 이러한 문제를 해결하기 위해 일회용으로 사용할 함수를 정의하는데 유용한 **람다함수** 기능을 지원합니다. 람다함수를 사용하면 위에서 `weather()` 함수를 정의하는 과정을 생략하고 다음과 같이 사용할 수 있습니다.

In [18]:
# 람다함수 쓰는 방법
df["날씨"]=df["날씨"].apply(lambda e : "눈/비" if e=="눈" or e=="비" else e)

In [33]:
# 어른, 청소년, 어린이의 평균값 구하기
df[['어른', '청소년', '어린이']].apply(np.average)
df[['어른', '청소년', '어린이']].apply(lambda x: np.average(x))

어른     3568.507366
청소년            NaN
어린이     550.260589
dtype: float64

In [34]:
# 청소년에 Nan 이 나온는데 결측치를 제거해주고 다시 계산해야함
df[df[['청소년']].notna()]   # 결측치가 아니면 True, 결측치이면 False

,날짜,공휴일,날씨,유료합계,어른,청소년,어린이,외국인,단체,무료합계,총계,연,월,일,요일
0,NaN,NaN,NaN,NaN,NaN,141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1082,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,NaN,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df[df['청소년'].notna()][['어른', '청소년', '어린이']].apply(np.average, axis=1)  # axis='columns'

0       1119.666667
1       1724.333333
2       1002.666667
4        138.666667
5        122.333333
           ...     
1081     168.000000
1082     253.666667
1083     548.000000
1084     513.000000
1085    1020.333333
Length: 1081, dtype: float64

### [TODO] 일차별 혼잡도 시리즈를 생성한 뒤 변수 `congestion`에 저장하세요.
* `apply()`를 사용하세요.
* "총계" 컬럼의 값이 8000 이상인 날은 "혼잡", 8000 미만인 날은 "원활" 로 합니다.
* 평범하게 함수를 정의해도 되고, 람다함수를 활용해도 됩니다.

In [37]:
# None을 지우고 알맞은 코드를 입력하세요.
congestion = df["총계"].apply(lambda x : "혼잡" if x >= 8000 else "원활")

## 채점
* **[TODO]** 중 수행하지 않은 부분이 없는지 확인하세요.
* 채점을 위해 아래 코드를 실행한 뒤 우측 상단의 제출 버튼을 눌러주세요.
* 코드 수정시 정상적인 채점이 이루어지지 않습니다.

In [38]:
import os
import json

congestion.to_json("result.json")